In [ ]:
# INSTALL PACKAGES: 

#%pip install pandas
#%pip install numpy
#%pip install numpyencoder 
#%pip install unicodedata 
#%pip install datetime
#%pip install matplotlib
#%pip install scikit-learn
#%pip install scipy
#%pip install nltk
#%pip install operator
#%pip install gensim
#%pip install spacy
#%pip install es_core_news_sm

# Ejemplo de DDR con Fast Text

Si quieren leer más, DDR está super bien explicado en este sitio:
https://github.com/USC-CSSL/DDR/blob/master/docs/DDR-Tutorial.rst#generate-document-dictionary-similarity-measures

In [2]:
##imports##
import pandas as pd
import numpy as np
import json
import os
from random import sample 
from numpyencoder import NumpyEncoder
import math
from unicodedata import normalize
from datetime import datetime  
from datetime import timedelta  
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from scipy.sparse import csr_matrix

# NLTK Stop words
import nltk
import nltk; nltk.download('stopwords')
from nltk.corpus import stopwords
import operator
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.keyedvectors import KeyedVectors
# spacy for lemmatization
import spacy
from spacy.lang.es import Spanish
stop_words = stopwords.words('spanish')
stop_words.extend(['i','ii','iii','iv','v','vi','vii', 'ix' , 'x' , 'xi'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aleja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
nlp  =  spacy.load("es_core_news_sm")

C:\Users\aleja\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ..\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


Vamos a cargar el modelo de Word Embeddings ya entrenado en español por FastText con textos de wikipedia (Esto puede consumir harta memoria, pq el archivo pesa más de 2Gb)

In [4]:
##Model Loading##
# embeddings-l-model.vec: Spanish Unannotated Corpora	2.6B	FastText	1,313,423	300 José Cañete
# wiki.es.vec:            Spanish Wikipedia	             ???	FastText	985,667	    300	FastText team

wordvectors_file_vec  =  'word_embeddings_models/wiki.es.vec'#'embeddings-l-model.vec'#
wordvectors  =  KeyedVectors.load_word2vec_format(wordvectors_file_vec)
model_word_set = list(wordvectors.index_to_key)

In [5]:
len(model_word_set)

985667

Vamos a definir una serie de funciones que serán útiles para procesar el texto (sacar stop-words, lemmatizar y esas cosas), para calcular los vectores representativos de un documento y para hacer los cálculos de similitud del coseno entre dos vectores

In [6]:
##Function definitions##

def cos_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc = True,max_len = 22))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc),max_len = 22,deacc = True) if word not in stop_words] for doc in texts]

def lemmatization(texts, allowed_postags = ['NOUN', 'ADJ', 'VERB', 'ADV','PROPN','X']):
    """https://spacy.io/api/annotation"""
    texts_out  =  []
    for sent in texts:
        doc  =  nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

def make_agg_vec(words, model, num_features, model_word_set, filter_out = []):
    """Create aggregate representation of list of words"""
    feature_vec  =  np.zeros((num_features,), dtype = "float32")
    nwords  =  0.0
    lost_words = []
    for word in words:
        if word not in filter_out:
            if word in model_word_set:
                nwords += 1
                feature_vec = np.add(feature_vec , model[word])
            else:
                lost_words.append(word)
    if nwords>0:
        avg_feature_vec = feature_vec / nwords
        return [avg_feature_vec,lost_words]
    else:
      return [feature_vec,lost_words]


def cos_similarity_distance(v1, v2):
 return 1-cos_similarity(v1,v2)



def cos_similarity_angle(v1, v2):
    prod  =  np.dot(v1, v2)
    len1  =  math.sqrt(np.dot(v1, v1))
    len2  =  math.sqrt(np.dot(v2, v2))
    if   abs(prod / (len1 * len2) -1)<(10**(-4)):
        return 0
    if len1*len2 == 0:
        return -1
    else:
      return   math.degrees(    math.acos( prod / (len1 * len2))  )# retorna un angulo entre 0 y 180
    


def cos_similarity_pearson(v1, v2):
    v1    = v1-np.mean(v1)
    v2    = v2-np.mean(v2)
    prod  =  np.dot(v1, v2)
    len1  =  math.sqrt(np.dot(v1, v1))
    len2  =  math.sqrt(np.dot(v2, v2))
    if len1*len2 == 0:
        return -1
    else:
      return   prod / (len1 * len2)  # retorna un angulo entre 0 y 180
    


def preprocess_document(doc):
    """ Preprocesar el documento eliminando stopwords, lematizando, eliminando caracteres no alfanuméricos y palabras vacías. """
    # Asegurarse de que el documento sea una cadena
    if isinstance(doc, list):
        doc = ' '.join(doc)
    # Eliminar stopwords y lematizar
    # doc = nlp(doc)
    # lemmatized = [token.lemma_ for token in doc if not token.is_stop]
    # print(doc)
    no_stopw=remove_stopwords([doc])
    # print(no_stopw)
    lemmatized=lemmatization(no_stopw)
    # Eliminar caracteres no alfanuméricos, convertir a minúsculas y eliminar palabras vacías
    # cleaned = [re.sub(r'\W+', '', word).lower() for word in lemmatized]
    cleaned = [word for word in lemmatized]  # Elimina las palabras vacías
    cleaned=cleaned[0]
    return cleaned




Ahora vamos a definir los diccionarios con los conceptos "Contenido" y "Presentación" cada uno representado por una serie de palabras.

In [7]:
# Diccionarios

content_test_dictionary = [
    "física",
    "energía",
    "átomo",
    "mecánica",
    "cuántica",
    "electromagnetismo",
    "termodinámica",
    "relatividad",
    "partículas",
    "óptica",
    "nuclear",
    "experimento",
    "teoría",
    "leyes",
    "fórmula",
    "ondas",
    "electrones",
    "gravitación",
    "espectro",
    "campo",
    "tarea",
    "ejercicio",
    "certamen",
    "laboratorio",
    "entrega", 
    "test",
    "informe",
    "evaluacion"
    ]
presentation_test_dictionary = [
    "gracias",
    "por favor",
    "disculpa",
    "perdon",
    "bienvenido",
    "encantado",
    "amigo",
    "companero",
    "equipo",
    "reunión",
    "evento",
    "saludos",
    "fiesta",
    "invitación",
    "sociedad",
    "familia",
    "grupo",
    "encuentro",
    "hora",
    "hola",
    "instagram"
        ]

Usaremos las siguientes frases para clasificarlas entre "Contenido" y "Presentación"

In [8]:
# Ejemplo de documentos
testmessage1 = ['hola, me llamo antonio y estoy ansioso por comenzar el curso']
testmessage2 = ['Los diagramas de cuerpo libre nos permiten visualizar que fuerzas actuan sobre un sistema']
testmessage3 = ['física, ondas energia gravitación nuclear óptica átom cuántica']
testmessage4 = ['hola, me llamo antonio y estoy ansioso por comenzar el curso. Los diagramas de cuerpo libre nos permiten visualizar que fuerzas actuan sobre un sistema']
testmessage5 = ['Los diagramas de cuerpo libre nos permiten visualizar que fuerzas actuan sobre un sistema. La ecuación de Navier-Stokes describe el comportamiento de los fluidos en movimiento. Estas ecuaciones son fundamentales en la dinámica de fluidos y son un conjunto de ecuaciones diferenciales parciales no lineales que describen el flujo de fluidos newtonianos.']

doc1_words = preprocess_document(testmessage1)
doc2_words = preprocess_document(testmessage2)
doc3_words = preprocess_document(testmessage3)
doc4_words = preprocess_document(testmessage4)
doc5_words = preprocess_document(testmessage5)

Ahora vamos a calcular el vector que representa a cada uno de los documentos, usando el modelo de word embeddings en cada una de las palabras del documento y promediándolas:

In [9]:
# Agregación Promedio: wordvectors es el modelo de Fast Text (los vectores) y model_word_set es el conjunto de palabras del modelo
vec_1_avg = make_agg_vec(doc1_words, wordvectors, 300, model_word_set)[0]
vec_2_avg = make_agg_vec(doc2_words, wordvectors, 300, model_word_set)[0]
vec_3_avg = make_agg_vec(doc3_words, wordvectors, 300, model_word_set)[0]
vec_4_avg = make_agg_vec(doc4_words, wordvectors, 300, model_word_set)[0]
vec_5_avg = make_agg_vec(doc5_words, wordvectors, 300, model_word_set)[0]

Vamos a hacer lo mismo para cada concepto:

In [10]:
vec_dp_avg = make_agg_vec(presentation_test_dictionary, wordvectors, 300, model_word_set)[0]
vec_dc_avg = make_agg_vec(content_test_dictionary, wordvectors, 300, model_word_set)[0]

Ahora vamos a calcular que tan cerca o lejos están cada documento de cada concepto, calculando el coseno del ángulo entre los vectores:

In [11]:
# Calcular la similitud (usando, por ejemplo, la similitud del coseno)
similarity_test1_content = cos_similarity(vec_1_avg, vec_dc_avg)
similarity_test2_content = cos_similarity(vec_2_avg, vec_dc_avg)
similarity_test3_content = cos_similarity(vec_3_avg, vec_dc_avg)
similarity_test4_content = cos_similarity(vec_4_avg, vec_dc_avg)
similarity_test5_content = cos_similarity(vec_5_avg, vec_dc_avg)

similarity_test1_presentacion = cos_similarity(vec_1_avg, vec_dp_avg)
similarity_test2_presentacion = cos_similarity(vec_2_avg, vec_dp_avg)
similarity_test3_presentacion = cos_similarity(vec_3_avg, vec_dp_avg)
similarity_test4_presentacion = cos_similarity(vec_4_avg, vec_dp_avg)
similarity_test5_presentacion = cos_similarity(vec_5_avg, vec_dp_avg)

In [12]:
print(testmessage1)
print('Similitud texto 1 - Presentacion:',cos_similarity(vec_1_avg, vec_dp_avg),'--','Similitud texto 1 - Contenido:',cos_similarity(vec_1_avg, vec_dc_avg))
print(testmessage2)
print('Similitud texto 2 - Presentacion:',cos_similarity(vec_2_avg, vec_dp_avg),'--','Similitud texto 2 - Contenido:',cos_similarity(vec_2_avg, vec_dc_avg))
print(testmessage3)
print('Similitud texto 2 - Presentacion:',cos_similarity(vec_3_avg, vec_dp_avg),'--','Similitud texto 2 - Contenido:',cos_similarity(vec_3_avg, vec_dc_avg))
print(testmessage4)
print('Similitud texto 2 - Presentacion:',cos_similarity(vec_4_avg, vec_dp_avg),'--','Similitud texto 2 - Contenido:',cos_similarity(vec_4_avg, vec_dc_avg))
print(testmessage5)
print('Similitud texto 2 - Presentacion:',cos_similarity(vec_5_avg, vec_dp_avg),'--','Similitud texto 2 - Contenido:',cos_similarity(vec_5_avg, vec_dc_avg))


['hola, me llamo antonio y estoy ansioso por comenzar el curso']
Similitud texto 1 - Presentacion: 0.7337279 -- Similitud texto 1 - Contenido: 0.45199558
['Los diagramas de cuerpo libre nos permiten visualizar que fuerzas actuan sobre un sistema']
Similitud texto 2 - Presentacion: 0.4917478 -- Similitud texto 2 - Contenido: 0.67510384
['física, ondas energia gravitación nuclear óptica átom cuántica']
Similitud texto 2 - Presentacion: 0.43267116 -- Similitud texto 2 - Contenido: 0.7785211
['hola, me llamo antonio y estoy ansioso por comenzar el curso. Los diagramas de cuerpo libre nos permiten visualizar que fuerzas actuan sobre un sistema']
Similitud texto 2 - Presentacion: 0.680647 -- Similitud texto 2 - Contenido: 0.664385
['Los diagramas de cuerpo libre nos permiten visualizar que fuerzas actuan sobre un sistema. La ecuación de Navier-Stokes describe el comportamiento de los fluidos en movimiento. Estas ecuaciones son fundamentales en la dinámica de fluidos y son un conjunto de ecua